# Step 2: Model Building & Evaluation

In [1]:
# import the libraries
from tensorflow import keras
from keras import layers
from keras.optimizers import schedules
#print(keras.__version__)

import h5py
import os
import pandas as pd
import numpy as np

import urllib
import glob
import pickle
import re

from sklearn.metrics import confusion_matrix, recall_score, precision_score
from keras.models import Sequential
from sklearn import datasets
from imblearn.under_sampling import RandomUnderSampler
#from keras.layers import Dense, Dropout, LSTM, Activation

print('Import complete successfully!')

Import complete successfully!


## Load feature data set

In [2]:
train_df = pd.read_csv('./data/preparedData/PM_train_prepared.csv', sep=",", header=0)
train_df.head(10)

,datetime,machineID,cycle,model,age,volt,rotate,pressure,vibration,RUL,label1,label2,cycle_norm
0,2015-01-01 06:00:00,1,1,0.666667,0.9,0.499928,0.503194,0.459050,0.487945,96,0,0,0.000000
1,2015-01-01 07:00:00,1,2,0.666667,0.9,0.415395,0.474884,0.328274,0.460912,95,0,0,0.000154
2,2015-01-01 08:00:00,1,3,0.666667,0.9,0.466796,0.698752,0.178160,0.311752,94,0,0,0.000309
3,2015-01-01 09:00:00,1,4,0.666667,0.9,0.412756,0.373197,0.430624,0.423896,93,0,0,0.000463
4,2015-01-01 10:00:00,1,5,0.666667,0.9,0.382001,0.533508,0.450207,0.179498,92,0,0,0.000617
5,2015-01-01 11:00:00,1,6,0.666667,0.9,0.476397,0.524429,0.331737,0.335594,91,0,0,0.000772
6,2015-01-01 12:00:00,1,7,0.666667,0.9,0.375322,0.647946,0.449235,0.450251,90,0,0,0.000926
7,2015-01-01 13:00:00,1,8,0.666667,0.9,0.476511,0.487240,0.369402,0.332800,89,0,0,0.001080
8,2015-01-01 14:00:00,1,9,0.666667,0.9,0.494267,0.467520,0.440837,0.494318,88,0,0,0.001235
9,2015-01-01 15:00:00,1,10,0.666667,0.9,0.455570,0.579276,0.397960,0.404184,87,0,0,0.001389


In [3]:
#balancing training data
#print('Balancing train data is in progress...')
df_train = train_df[["datetime","machineID","cycle","model","age","volt","rotate","pressure","vibration","RUL","label2","cycle_norm"]]
y_train = train_df["label1"]
rus = RandomUnderSampler()
#rus = RandomUnderSampler(sampling_strategy={0:30000})
df_resampled, y_resampled = rus.fit_resample(df_train, y_train)
df_balanced = pd.concat([df_resampled, y_resampled],axis=1)
train_df = df_balanced.sort_values(['machineID', 'datetime'], ascending=[True, True])
train_df = train_df[['datetime','machineID', 'cycle', 'model', 'age', 'volt', 'rotate', 'pressure', 'vibration', 'RUL', 'cycle_norm', 'label1', 'label2']]
train_df.drop(train_df.columns[[0]], axis=1, inplace=True)
train_df.head(10)
#print('Balancing train data completed successfully.')

,machineID,cycle,model,age,volt,rotate,pressure,vibration,RUL,cycle_norm,label1,label2
3620,1,2,0.666667,0.9,0.415395,0.474884,0.328274,0.460912,95,0.000154,0,0
6412,1,53,0.666667,0.9,0.616306,0.556835,0.350024,0.577011,44,0.008026,0,0
19935,1,67,0.666667,0.9,0.620057,0.534940,0.390426,0.566431,30,0.010187,1,1
19936,1,68,0.666667,0.9,0.453993,0.590838,0.489264,0.702829,29,0.010341,1,1
19937,1,69,0.666667,0.9,0.430023,0.604926,0.470232,0.689213,28,0.010495,1,1
19938,1,70,0.666667,0.9,0.600360,0.661463,0.463462,0.693415,27,0.010650,1,1
19939,1,71,0.666667,0.9,0.622540,0.441451,0.410613,0.561752,26,0.010804,1,1
19940,1,72,0.666667,0.9,0.464634,0.585275,0.389755,0.703889,25,0.010958,1,1
19941,1,73,0.666667,0.9,0.428900,0.557030,0.344969,0.538842,24,0.011113,1,1
19942,1,74,0.666667,0.9,0.287297,0.529740,0.501922,0.645600,23,0.011267,1,1


In [4]:
test_df = pd.read_csv('./data/preparedData/PM_test_prepared.csv', sep=",", header=0)
test_df.head(10)

,machineID,cycle,model,age,volt,rotate,pressure,vibration,RUL,cycle_norm,label1,label2
0,89,1,0.666667,0.833333,0.420252,0.816190,0.653248,0.549285,888,0.000000,0,0
1,89,2,0.666667,0.833333,0.408611,0.762715,0.829219,0.476602,887,0.000198,0,0
2,89,3,0.666667,0.833333,0.250609,0.301397,0.657232,0.357829,886,0.000397,0,0
3,89,4,0.666667,0.833333,0.497216,0.554726,0.809499,0.160269,885,0.000595,0,0
4,89,5,0.666667,0.833333,0.418949,0.471659,0.482283,0.490419,884,0.000794,0,0
5,89,6,0.666667,0.833333,0.523158,0.518877,0.767339,0.514914,883,0.000992,0,0
6,89,7,0.666667,0.833333,0.457698,0.741963,0.429188,0.376204,882,0.001191,0,0
7,89,8,0.666667,0.833333,0.437202,0.455160,0.392089,0.416383,881,0.001389,0,0
8,89,9,0.666667,0.833333,0.477320,0.468083,0.469835,0.461749,880,0.001588,0,0
9,89,10,0.666667,0.833333,0.492070,0.599858,0.345500,0.507347,879,0.001786,0,0


## Modelling

In [5]:
# pick a large window size of 120 cycles
sequence_length = 120

In [6]:
# function to reshape features into (samples, time steps, features) 
def gen_sequence(id_df, seq_length, seq_cols):
    """ Only sequences that meet the window-length are considered, no padding is used. This means for testing
    we need to drop those which are below the window-length. An alternative would be to pad sequences so that
    we can use shorter ones """
    data_array = id_df[seq_cols].values
    num_elements = data_array.shape[0]
    for start, stop in zip(range(0, num_elements-seq_length), range(seq_length, num_elements)):
        yield data_array[start:stop, :]

In [7]:
# pick the feature columns 
sequence_cols = ['cycle_norm']
key_cols = ['machineID', 'cycle']
label_cols = ['label1', 'label2', 'RUL']

input_features = test_df.columns.values.tolist()
sensor_cols = [x for x in input_features if x not in set(key_cols)]
sensor_cols = [x for x in sensor_cols if x not in set(label_cols)]
sensor_cols = [x for x in sensor_cols if x not in set(sequence_cols)]

# The time is sequenced along
# This may be a silly way to get these column names, but it's relatively clear
sequence_cols.extend(sensor_cols)

print(sensor_cols)
print(sequence_cols)

['model', 'age', 'volt', 'rotate', 'pressure', 'vibration']
['cycle_norm', 'model', 'age', 'volt', 'rotate', 'pressure', 'vibration']


In [8]:
# generator for the sequences
seq_gen = (list(gen_sequence(train_df[train_df['machineID']==id], sequence_length, sequence_cols)) 
           for id in train_df['machineID'].unique())

seq_array = np.concatenate(list(seq_gen)).astype(np.float32)
seq_array.shape

(29070, 120, 7)

In [9]:
# function to generate labels
def gen_labels(id_df, seq_length, label):
    data_array = id_df[label].values
    num_elements = data_array.shape[0]
    return data_array[seq_length:num_elements, :]

In [10]:
# generate labels
label_gen = [gen_labels(train_df[train_df['machineID']==id], sequence_length, ['label1']) 
             for id in train_df['machineID'].unique()]
label_array = np.concatenate(label_gen).astype(np.float32)
label_array.shape

(29070, 1)

## Transformer model

In [11]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Attention and Normalization
    x = layers.MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=dropout)(inputs, inputs)
    x = layers.Dropout(dropout)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(res)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    return x + res

In [12]:
# build the model

input_shape = seq_array.shape[1:]
n_classes = 1 #len(np.unique(label_array))
#nb_features = seq_array.shape[2]
#nb_out = label_array.shape[1]
head_size=256
num_heads=8
ff_dim=8
num_transformer_blocks=6
mlp_units=[256]
mlp_dropout=0.4
dropout=0.25

inputs = keras.Input(shape=input_shape)
x = inputs
for _ in range(num_transformer_blocks):
    x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)
x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
for dim in mlp_units:
    x = layers.Dense(dim, activation="relu")(x)
    x = layers.Dropout(mlp_dropout)(x)
outputs = layers.Dense(n_classes, activation="sigmoid")(x)

model = keras.Model(inputs, outputs)

#lr_schedule = 1e-4
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-2,
    decay_steps=20,
    decay_rate=0.9)

model.compile(loss="binary_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),
    metrics=["binary_accuracy"])

print(model.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 120, 7)]     0           []                               
                                                                                                  
 multi_head_attention (MultiHea  (None, 120, 7)      63495       ['input_1[0][0]',                
 dAttention)                                                      'input_1[0][0]']                
                                                                                                  
 dropout (Dropout)              (None, 120, 7)       0           ['multi_head_attention[0][0]']   
                                                                                                  
 layer_normalization (LayerNorm  (None, 120, 7)      14          ['dropout[0][0]']            

In [13]:
%%time
# fit the network
callbacks = [keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', # Monitor the validation binary accuracy
                                                     min_delta=0,    # until it doesn't change (or gets worse)
                                                     patience=8,  # patience > 1 so it continutes if it is not consistently improving
                                                     verbose=0, 
                                                     mode='auto',
                                                     restore_best_weights=True)]

model.fit(seq_array, # Training features
          label_array, # Training labels
          validation_split=0.3,
          epochs=50,
          batch_size=1024,
          verbose=1,
          callbacks=callbacks)

Epoch 1/50
20/20 [==============================] - 8881s 447s/step - loss: 0.7176 - binary_accuracy: 0.5187 - val_loss: 0.6943 - val_binary_accuracy: 0.5177
Epoch 2/50
20/20 [==============================] - 8853s 446s/step - loss: 0.6881 - binary_accuracy: 0.5498 - val_loss: 0.6891 - val_binary_accuracy: 0.5371
Epoch 3/50
20/20 [==============================] - 8855s 445s/step - loss: 0.6780 - binary_accuracy: 0.5719 - val_loss: 0.6742 - val_binary_accuracy: 0.5697
Epoch 4/50
20/20 [==============================] - 8841s 445s/step - loss: 0.6553 - binary_accuracy: 0.6082 - val_loss: 0.6672 - val_binary_accuracy: 0.6197
Epoch 5/50
20/20 [==============================] - 8813s 444s/step - loss: 0.6435 - binary_accuracy: 0.6257 - val_loss: 0.5728 - val_binary_accuracy: 0.7112
Epoch 6/50
20/20 [==============================] - 8843s 445s/step - loss: 0.5877 - binary_accuracy: 0.6857 - val_loss: 0.5257 - val_binary_accuracy: 0.7401
Epoch 7/50
20/20 [==============================] - 

In [14]:
# training metrics
scores = model.evaluate(seq_array, label_array, verbose=1, batch_size=200)
print('Training Accurracy: {}'.format(scores[1]))

146/146 [==============================] - 3606s 25s/step - loss: 0.4489 - binary_accuracy: 0.7933
Training Accurracy: 0.7932920455932617


In [15]:
# make predictions and compute confusion matrix
y_pred = (model.predict(seq_array,verbose=1, batch_size=200) > 0.5).astype("int32")
y_true = label_array
print('Training Confusion matrix\n- x-axis is true labels.\n- y-axis is predicted labels')
cm = confusion_matrix(y_true, y_pred)
cm

146/146 [==============================] - 3634s 25s/step
Training Confusion matrix
- x-axis is true labels.
- y-axis is predicted labels


array([[11054,  3129],
       [ 2880, 12007]], dtype=int64)

In [16]:
# compute precision and recall
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = 2 * (precision * recall) / (precision + recall)
print( 'Training Precision: ', precision, '\n', 'Training Recall: ', recall, '\n', 'Training F1 Score:', f1)

Training Precision:  0.7932743128964059 
 Training Recall:  0.8065426210787936 
 Training F1 Score: 0.7998534456916364


## Model testing

In [17]:
seq_array_test_last = [test_df[test_df['machineID']==id][sequence_cols].values[-sequence_length:] 
                       for id in test_df['machineID'].unique() if len(test_df[test_df['machineID']==id]) >= sequence_length]

seq_array_test_last = np.asarray(seq_array_test_last).astype(np.float32)
seq_array_test_last.shape

(8, 120, 7)

In [18]:
y_mask = [len(test_df[test_df['machineID']==id]) >= sequence_length for id in test_df['machineID'].unique()]

label_array_test_last = test_df.groupby('machineID')['label1'].nth(-1)[y_mask].values
label_array_test_last = label_array_test_last.reshape(label_array_test_last.shape[0],1).astype(np.float32)
label_array_test_last.shape

print(seq_array_test_last.shape)
print(label_array_test_last.shape)

(8, 120, 7)
(8, 1)


In [19]:
# test metrics
scores_test = model.evaluate(seq_array_test_last, label_array_test_last, verbose=2)
print('Test Accurracy: {}'.format(scores_test[1]))

1/1 - 1s - loss: 0.7034 - binary_accuracy: 0.8750 - 1s/epoch - 1s/step
Test Accurracy: 0.875


In [20]:
# make predictions and compute confusion matrix
y_pred_test = (model.predict(seq_array_test_last,verbose=1, batch_size=200) > 0.5).astype("int32")
y_true_test = label_array_test_last
print('Confusion matrix\n- x-axis is true labels.\n- y-axis is predicted labels')
cm = confusion_matrix(y_true_test, y_pred_test)
cm

1/1 [==============================] - 1s 939ms/step
Confusion matrix
- x-axis is true labels.
- y-axis is predicted labels


array([[0, 1],
       [0, 7]], dtype=int64)

In [21]:
# compute precision and recall
precision_test = precision_score(y_true_test, y_pred_test)
recall_test = recall_score(y_true_test, y_pred_test)
f1_test = 2 * (precision_test * recall_test) / (precision_test + recall_test)
print( 'Test Precision: ', precision_test, '\n', 'Test Recall: ', recall_test, '\n', 'Test F1 Score:', f1_test)

Test Precision:  0.875 
 Test Recall:  1.0 
 Test F1 Score: 0.9333333333333333


## Saving or loading the model

In [22]:
# Save the model for operationalization: https://machinelearningmastery.com/save-load-keras-deep-learning-models/
import os
import h5py
from sklearn import datasets 

model.save('models/machineRULPredictorTramsformer.h5')
print("Model saved")

Model saved


In [ ]:
model = keras.models.load_model('models/machineRULPredictorTramsformer.h5')
print("Model loaded")